In [2]:
from neo4j import GraphDatabase
import pandas as pd

In [3]:
driver = GraphDatabase.driver(uri = "bolt://localhost:7687", auth = ("neo4j","heart"))

### Query: Find All of the Protein Targets that are Shared Between 2 Nodes

In [10]:
def search_drug_intersections(tx, category: str) -> list:
    query = ("MATCH (c:Category) WHERE c.category = $category "
             "MATCH (c)--(d:Drug)--(p) "
             "MATCH (p)-[:ASSOCIATED_WITH]->(r) "
             "RETURN p.protein, r.ReactomeID")
    result = tx.run(query, category = category)
    return result.data()

def run(category: str) -> pd.DataFrame:
    result = driver.session().write_transaction(search_drug_intersections, category)
    result = pd.DataFrame(result)
    return result

prompt = input("Enter Drug Category:")
print("Drug Targets and ReactiomeIDs for: ", prompt)
run(prompt)

Drug Targets and ReactiomeIDs for:  Na Channel Blockers


,p.protein,r.ReactomeID
0,Q14524,R-HSA-445095
1,Q14524,R-HSA-5576892
2,P07550,R-HSA-9660821
3,P07550,R-HSA-8856828
4,P07550,R-HSA-8856825
...,...,...
188,P26358,R-HSA-4655427
189,P26358,R-HSA-427413
190,P26358,R-HSA-212300
191,Q14525,R-HSA-6809371


### Find Shared PMIDs Between Drug Categories

In [25]:
def search_drug_intersections(tx, category1: str, category2: str, limit: int) -> list:
    query = ("MATCH (c:Category) WHERE c.category = $category1 "
             "MATCH (f:Category) WHERE f.category = $category2 "
             "MATCH (c)--(d:Drug)--(p) "
             "MATCH (f)--(l:Drug)--(w)"
             "MATCH (p)-[:PMID_TARGET]->(t)<-[:PMID_TARGET]-(w) "
             "RETURN t.id LIMIT $limit")
    result = tx.run(query, category1 = category1, category2 = category2, limit = limit)
    return result.data()

def run(category1: str, category2: str, limit: int) -> pd.DataFrame:
    result = driver.session().write_transaction(search_drug_intersections, category1, category2, limit)
    result = pd.DataFrame(result)
    return result

prompt1 = input("Enter Drug Category 1:")
prompt2 = input("Enter Drug Category 2:")
prompt3 = input("Limit: ")
print("Shared PMIDs for: ", prompt1, "and", prompt2, "LIMIT:", prompt3)
run(prompt1, prompt2, int(prompt3))

Shared PMIDs for:  Beta Blockers and Diuretics LIMIT: 10


,t.id
0,24155982
1,28204831
2,28740174
3,25863800
4,20174578
5,26229964
6,30571258
7,30571503
8,23940716
9,24155982


-------

### Node Similarity Algorithm

@cypher:

    CALL gds.graph.create(
    'drug-protein',
    ['Drug', 'Protein', 'Pathway'], 
    {
      TARGET: {orientation: 'UNDIRECTED'}, ASSOCIATED_WITH: {orientation: 'UNDIRECTED'}
    }
    )

In [139]:
class NodeSimilarity():
    """Class to run node sim"""
    def __init__(self) -> None:
        self.driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "heart"))

    def close(self) -> None:
        self.driver.close()

    @classmethod
    def nodesim(cls, tx) -> any:
        """
        @param cls is the class
        @param tx is the transaction
        @return result.data() is the data of the cluster
        """
        query = ("""
        CALL gds.nodeSimilarity.stream('drug-protein')
        YIELD node1, node2, similarity
        RETURN gds.util.asNode(node1).name AS dbid1, gds.util.asNode(node2).name AS dbid2, similarity
        ORDER BY similarity DESCENDING, dbid1, dbid1
        """)
        result = tx.run(query)
        return result.data()


    def run_nodesim(self) -> any:
        """
        @param self
        @return result is the dataframe from the pagerank
        """
        result = self.driver.session().write_transaction(self.nodesim)
        result = pd.DataFrame(result)
        return result

nodesim = NodeSimilarity()
result = nodesim.run_nodesim()
result = result[result['similarity'] == 1]
result

,dbid1,dbid2,similarity
0,Canagliflozin,None,1.0
1,Canagliflozin,None,1.0
2,Insulin,colestipol,1.0
3,Insulin,bucindolol,1.0
4,Insulin,dopexamine,1.0
...,...,...,...
2789,None,None,1.0
2790,None,None,1.0
2791,None,None,1.0
2792,None,None,1.0


In [142]:
result[:248].to_csv("Node Similarity")

### Pagerank Algorithm

In [158]:
class pagerank():
    """Class to run pageranke"""
    def __init__(self) -> None:
        self.driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "heart"))

    def close(self) -> None:
        self.driver.close()

    @classmethod
    def pagerank(cls, tx) -> any:
        """
        @param cls is the xaclass
        @param tx is the transaction
        @return result.data() is the data of the pagerank
        """
        query = ("Call gds.pageRank.stream('drug-protein') YIELD nodeId, score RETURN gds.util.asNode(nodeId).protein AS UniProtID, score ORDER BY score DESC, UniProtID ASC")
        result = tx.run(query)
        return result.data()


    def run_pagerank(self) -> any:
        """
        @param self
        @return result is the dataframe from the pagerank
        """
        result = self.driver.session().write_transaction(self.pagerank)
        result = pd.DataFrame(result)
        return result

In [159]:
pagerank = pagerank()

In [161]:
result = pagerank.run_pagerank()
result[:196].to_csv("Pagerank_for_unid.csv")

In [157]:
result[:146]

,UniProtID,score
0,None,11.168309
1,None,11.161981
2,None,10.781645
3,None,10.442585
4,None,9.910084
...,...,...
141,None,1.684088
142,None,1.681592
143,None,1.680469
144,None,1.670963


In [137]:
result[:146].to_csv("Pagerank")